In [ ]:
import xarray as xr
from skimage.morphology import skeletonize
from shapely.geometry import MultiPolygon
import geopandas as gpd
import rioxarray as rxr
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
gen_path = '/Users/emarshall/Desktop/siparcs/xr_book1/'

In [ ]:
gen_path_home = '/Users/emmamarshall/Desktop/phd_research/siparcs/'
rgi_path_home = '/Users/emmamarshall/Downloads/15rgi60SouthAsiaEast'

## Read in vector data

In [ ]:
#se_asia = gpd.read_file(gen_path_home[:-9] + 'data/nsidc0770_15.rgi60.SouthAsiaEast/15_rgi60_SouthAsiaEast.shp')
se_asia = gpd.read_file(rgi_path_home + '/15_rgi60_SouthAsiaEast.shp')


In [ ]:
se_asia_prj = se_asia.to_crs('+proj=lcc +lat_1=15 +lat_2=65 +lat_0=30 +lon_0=95 +x_0=0 +y_0=0 +ellps=WGS84 +datum=WGS84 +units=m no_defs')
se_asia_prj

In [ ]:
sample_glacier_prj = se_asia_prj.loc[se_asia_prj['RGIId'] == 'RGI60-15.11909']
sample_glacier_prj

In [ ]:
sample_glacier_series_prj = se_asia_prj.iloc[11908]

In [ ]:
import shapely
shapely.__version__

In [ ]:
attributes = {'id': 11909, 'name':'sample_glacier','valid':True}
centerline = Centerline(sample_glacier_series_prj['geometry'], **attributes)

In [ ]:
type(centerline)
centerline.name

## Read in raster data 
- for once we've exctracted centerline


In [ ]:
itslive = rxr.open_rasterio(gen_path_home + 'HMA_G0120_0000.nc').squeeze()

In [ ]:
glacier_raster = itslive.rio.clip(sample_glacier_prj.geometry, sample_glacier_prj.crs)

In [ ]:
#get rid of non ice pixels (turn to nan)
just_ice = xr.where(glacier_raster.ice == 1., glacier_raster, np.NaN)

In [ ]:
fig, ax = plt.subplots(figsize=(9,9))

just_ice.v.plot(ax=ax)

## Extract glacier centerline (trying to in a more xarray-like way)

In [ ]:
from skimage.morphology import skeletonize 

In [ ]:
centerline_test = just_ice.ice.reduce(skeletonize)
centerline_test.plot()
type(centerline_test)

In [ ]:
centerline_test_ds = glacier_raster.where(centerline_test != False)
centerline_test_ds.v.plot()

In [ ]:
glacier_dict = {'full':glacier_raster,
                'clipped_to_ice': just_ice,
                'centerline': centerline_test_ds}

In [ ]:
glacier_dict['clipped_to_ice']

## Extract glacier centerline (original way)

In [ ]:
skeleton_ice = skeletonize(just_ice.ice.data)
skeleton_ice_lee = skeletonize(just_ice.ice.data)

Compare different skeletonize algorithms:

In [ ]:
fig, axes = plt.subplots(ncols=3, figsize = (14,4), sharex=True, sharey=True)

ax = axes.ravel()

ax[0].imshow(just_ice.v, cmap=plt.cm.cividis)
ax[0].axis('off')
ax[0].set_title('Glacier velocity', fontsize=14)
ax[0].imshow(skeleton_ice, cmap=plt.cm.Reds, alpha = 0.2)
ax[0].imshow(skeleton_ice_lee, cmap=plt.cm.Greens, alpha = 0.2)


ax[1].imshow(skeleton_ice, cmap=plt.cm.Reds)
ax[1].axis('off')
ax[1].set_title('skeleton', fontsize=14)

ax[2].imshow(skeleton_ice_lee, cmap=plt.cm.Greens)
ax[2].axis('off')
ax[2].set_title('skeleton (lee)', fontsize=14)

fig.tight_layout()
fig.show

## Make xarray object out of centerline bool mask
- should have the same coords as the velocity xarray object

In [ ]:
centerline_da = xr.DataArray(
    skeleton_ice,
    dims=("y", "x"),
    coords={
        "x": glacier_raster.coords['x'].data,
        "y": glacier_raster.coords['y'].data}
)


centerline_da.plot()

In [ ]:
centerline_da

### Extract itslive xarray dataset data only at points along centerline
- this should return a dataset with the same data vars but only data along centerline

In [ ]:
centerline_ds = glacier_raster.where(da != False)

In [ ]:
centerline_ds

In [ ]:
fig, axs = plt.subplots(ncols = 2, figsize=(20,8))
just_ice.ice.plot(ax=axs[0], cmap=plt.cm.cividis)
centerline_ds.v.plot(ax=axs[0])

just_ice.v.plot(ax=axs[1])

### Experimenting with skeletonize v. thin (scikit image)

In [ ]:
just_ice.ice.plot()

In [ ]:
from skimage.morphology import skeletonize, thin

thinned = thin(just_ice.ice)
#thinned_partial = thin(just_ice.ice.data, max_num_iter = 50)

plt.imshow(thinned)

In [ ]:
fig, axs = plt.subplots(ncols=2)

axs[0].imshow(thinned, cmap=plt.cm.gray)
axs[0].set_title('thinned')
axs[0].axis('off')

axs[1].imshow(thinned_partial, cmap=plt.cm.gray)
axs[1].set_title('thinned partial')
axs[1].axis('off')

